#### Step 6: Example queries

What use is a data engineering project without actually asking some questions about our data? Below are some example queries (run against the local parquet files since I don't want to pay for S3 read costs).

##### Setup, imports and database loads

In [1]:
from setup import create_spark_session

spark = create_spark_session()

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SQLContext

from sql.exampleQueries import *

In [3]:
from etl import load_all_tables

time_dim_df, county_dim_df, state_dim_df, county_facts_df, state_facts_df = load_all_tables(spark)

time_dim_df.createOrReplaceTempView("dim_time")
county_dim_df.createOrReplaceTempView("dim_county")
state_dim_df.createOrReplaceTempView("dim_state")
county_facts_df.createOrReplaceTempView("fact_county")
state_facts_df.createOrReplaceTempView("fact_state")

sqlContext = SQLContext(spark)

Started loading database
Started loading time dimension table
Finished loading time dimension table
Started loading county dimension table
Finished loading county dimension table
Started loading state dimension table
Finished loading state dimension table
Started loading county facts table
Finished loading county facts table
Started loading state facts table
Finished loading state facts table
Finished loading database


##### Queries
Let's start simply. Which counties have the highest case rates?

In [ ]:
sqlContext.sql("""
    SELECT fc.fips, dc.county_name, max(fc.covid_case_total) as covid_case_total
        FROM fact_county fc
        LEFT JOIN dim_county dc
        ON fc.fips == dc.fips
        GROUP BY fc.fips, dc.county_name
        ORDER BY covid_case_total DESC
        LIMIT 10
""").show()

Which states have the most deaths in total?

In [ ]:
sqlContext.sql("""
    SELECT fs.state, max(fs.covid_case_total) as covid_case_total
        FROM fact_state fs
        GROUP BY fs.state
        ORDER BY covid_case_total DESC
        LIMIT 10
""").show()

Both of these are a bit misleading, though. Counties with higher population numbers are bound to have higher rates, so let's run these queries again but normalise them to the county/state population.

In [ ]:
sqlContext.sql("""
    WITH county_norm AS (
        SELECT fc.fips, fc.covid_case_total / dc.population as norm_case_total
        FROM fact_county fc
        LEFT JOIN dim_county dc
        ON fc.fips == dc.fips
        GROUP BY fc.fips, fc.covid_case_total, dc.population
    )
    SELECT fc.fips, dc.county_name, dc.state, max(cn.norm_case_total) as norm_case_total
        FROM fact_county fc
        LEFT JOIN dim_county dc
        ON fc.fips == dc.fips
        LEFT JOIN county_norm cn
        ON fc.fips == cn.fips
        GROUP BY fc.fips, dc.county_name, dc.state
        ORDER BY norm_case_total DESC
        LIMIT 10
""").show()

In [4]:
sqlContext.sql("""
    WITH state_norm AS (
        SELECT fs.state, fs.covid_death_total / ds.population as norm_death_total
        FROM fact_state fs
        LEFT JOIN dim_state ds
        ON fs.state == ds.state
        GROUP BY fs.state, fs.covid_death_total, ds.population
    )
    SELECT fs.state, max(sn.norm_death_total) as norm_death_total
        FROM fact_state fs
        LEFT JOIN state_norm sn
        ON fs.state == sn.state
        GROUP BY fs.state
        ORDER BY norm_death_total DESC
        LIMIT 10
""").show()

+-------------+--------------------+
|        state|    norm_death_total|
+-------------+--------------------+
|   New Jersey| 0.00260851409661762|
|     New York|0.002414721897611473|
|Massachusetts|0.002326376900875365|
| Rhode Island|0.002315298657448348|
|  Mississippi|0.002233026288033...|
|      Arizona|0.002226406601775...|
| South Dakota|0.002137752412905...|
|  Connecticut|0.002130622378532552|
|    Louisiana|0.002057305849941...|
|      Alabama|0.002031559343526046|
+-------------+--------------------+



Now for something a bit more tricky: Which counties have the largest temperature swings during the day?

In [6]:
sqlContext.sql("""
    SELECT fc.fips, dc.county_name, dc.state, fc.max_temp - fc.min_temp as temp_change
        FROM fact_county fc
        LEFT JOIN dim_county dc
        ON fc.fips == dc.fips
        GROUP BY fc.fips, dc.county_name, dc.state, fc.max_temp, fc.min_temp
        ORDER BY temp_change DESC
        LIMIT 10
""").show()

+-----+-----------+----------+------------------+
| fips|county_name|     state|       temp_change|
+-----+-----------+----------+------------------+
| 8031|     Denver|  Colorado|             32.97|
| 8014| Broomfield|  Colorado|32.940000000000005|
| 8001|      Adams|  Colorado|             32.75|
| 8005|   Arapahoe|  Colorado|              32.4|
| 8087|     Morgan|  Colorado|             32.25|
|40025|   Cimarron|  Oklahoma|             32.19|
|48431|   Sterling|     Texas|             32.04|
| 8039|     Elbert|  Colorado|31.980000000000004|
|56021|    Laramie|   Wyoming|             31.98|
|35059|      Union|New Mexico|31.799999999999997|
+-----+-----------+----------+------------------+



Looks like Colorado is seeing quite a few temperature changes! It might be interesting to group this by state, or to narrow the query down a bit since it's now looking only at the most extreme temperature, not at the average deviation every day.

I'd like to know how strongly Covid case rates correlate to population density. My guess is that high population density results in higher infection rates. To check this, we'll need to get the total case count for each county and normalise it for the total population, then compare the various percentiles of population density in the data set. We could split the whole dataset into 10% buckets of population density and calculate the average normalised case rate.

In [10]:
sqlContext.sql("""
    WITH percentiles AS (
        SELECT dc.fips, PERCENT_RANK() OVER(
            ORDER BY dc.population_density ASC
        ) AS percent_rank
        FROM dim_county dc
    ),
    percent_buckets AS (
        SELECT p.fips, ROUND(p.percent_rank, 1) AS bucket
        FROM percentiles p
    ),
    max_cases AS (
        SELECT fc.fips, max(fc.covid_case_total / dc.population_density) as normalised_covid_cases
        FROM fact_county fc
        LEFT JOIN dim_county dc
        ON fc.fips == dc.fips
        GROUP BY fc.fips
    )
    SELECT pb.bucket, avg(mc.normalised_covid_cases) as average_normalised_cases
        FROM fact_county fc
        LEFT JOIN percent_buckets pb
        ON fc.fips == pb.fips
        LEFT JOIN max_cases mc
        ON fc.fips == mc.fips
        GROUP BY pb.bucket
        ORDER BY pb.bucket DESC
        LIMIT 10
""").show()

+------+------------------------+
|bucket|average_normalised_cases|
+------+------------------------+
|   1.0|       34.96272359394068|
|   0.9|        57.3299859120545|
|   0.8|       53.69791338446918|
|   0.7|        68.2151273536616|
|   0.6|       55.55887627147644|
|   0.5|       64.02359784609416|
|   0.4|       67.70472271546825|
|   0.3|       76.53304705111098|
|   0.2|      118.82131288623005|
|   0.1|      169.38397129450493|
+------+------------------------+



This didn't turn out how I expected, which is exciting! It looks like the top 10% most densely populated counties actually have a much lower case rate considering the overall population than the next 10%, then it stabilises throughout the middle and the bottom 10% are much, much worse off than the average.

I'm not 100% confident that my query is correct, but this pretty much turned out exactly opposite what one would expect. I'd love to dig in more to evaluate factors that explain this result, maybe in a future project.

I'm also curious about how strongly a county's average temperature correlates with case rates. I'd assume that warmer counties fare worse.

A harder question to answer is how much weather on any given day affects covid case rates a few days down the line.  
Since the reporting won't be extremely accurate, we should limit ourselves to evaluating strings of consistently good/bad weather for a few days, and then look at the new case increase a week afterwards.

First, we need to classify what makes a day "good" or "bad". I'd define a "good" day as having low chance of precipitation and low cloud cover, whereas a "bad" day is high on both. We could include temperature here as well, but we'd need to look at a rolling average to see how any given day compares; this presents a problem since rolling averages don't play well with the idea of having a period of good/bad days (they're subsequent days, so the last day in the sequence would need e.g. a higher temperature than the preceding days even though they were classed as "good" already).

Next, we need to indentify sufficiently long sequences of days with similar weather.  
Then we can determine the cases a week from each day, and track the delta.

Finally, we can average the delta for good/bad weather days and see if we can find any difference.